# Fiona－地图文件读写Python库

Fiona提供几何对象的文件读写功能，是基于OGR实现的简洁的Python API对象接口，详情可以查看[文档](http://toblerity.org/fiona/README.html#usage)和[源代码](https://github.com/Toblerity/Fiona)、[例程](https://github.com/Toblerity/Fiona/tree/master/examples)。

Fiona 是 OGR的简洁、灵活, 规范的 Python API。Fiona 设计为简单而可靠的支持库，聚焦在使用标准的Python IO风格进行数据的读写，使用Python的list/dict等方法代替OGR的类对象访问。Fiona 支持多层的GIS数据格式的读写，以及压缩的虚拟文件系统并且与其它的Python GIS软件整合到一起，如Shapely、pyproj、Rtree等。

## 用法

### Collections

从fiona.open()函数中返回Collection对象，可以进行记录（Records）的读写。Records被映射为GeoJSON格式。Fiona本身不包含任何空间操作方法，几何和空间操作可以通过Shapely或类似的库来实现。下面是一个例子：

函数 fiona.drivers() 和 context manager 在 1.1版本中引入。上面的例子显示了register and de-register drivers的方法。Fiona 1.0 仍然可以工作: 打开的collections管理global driver registry，如果没有其它的manager。

### Reading Multilayer data

Collections可以从单个的layers中创建，Layer可以位于多层的文件或目录。 目标layer通过名称或索引号进行指定。函数fiona.listlayers()返回给定文件或目录的所有层的列表，如下所示。

    with fiona.drivers():

        for layername in fiona.listlayers('docs/data'):
            with fiona.open('docs/data', layer=layername) as c:
                print(layername, len(c))

    # Output:
    # test_uk 48

Layer 可以按照index指定。在这里，layer=0 和 layer='test_uk' 指定的是同一个layer。如下所示：

    with fiona.drivers():

        for i, layername in enumerate(fiona.listlayers('docs/data')):
            with fiona.open('docs/data', layer=i) as c:
                print(i, layername, len(c))

    # Output:
    # 0 test_uk 48

### Writing Multilayer data

Multilayer数据可以同样地被写入。写入时，Layers 必须按照名称指定。下面写入一个shape文件:

    with fiona.drivers():

        with open('docs/data/test_uk.shp') as c:
            meta = c.meta
            f = next(c)

        with fiona.open('/tmp/foo', 'w', layer='bar', **meta) as c:
            c.write(f)

        print(fiona.listlayers('/tmp/foo'))
        # Output: ['bar']

        with fiona.open('/tmp/foo', layer='bar') as c:
            print(len(c))
            f = next(c)
            print(f['geometry']['type'])
            print(f['properties'])

        # Output:
        # 1
        # Polygon
        # {'FIPS_CNTRY': 'UK', 'POP_CNTRY': 60270708.0, 'CAT': 232.0,
        #  'AREA': 244820.0, 'CNTRY_NAME': 'United Kingdom'}

所创建的文件在 /tmp/foo 目录，如下：

    $ ls /tmp/foo
    
    bar.cpg bar.dbf bar.prj bar.shp bar.shx

### 虚拟文件系统和压缩包

Zip 和 Tar 归档文件可以被作为虚拟文件系统对待处理。换句话说，Fiona可以支持压缩的ShapeFiles的读写。如下所示：

    with fiona.drivers():

        for i, layername in enumerate(
            fiona.listlayers('/',vfs='zip://docs/data/test_uk.zip')):
            
            with fiona.open(
                '/',vfs='zip://docs/data/test_uk.zip',layer=i) as c:
                
                print(i, layername, len(c))

    # Output:
    # 0 test_uk 48

## Fiona CLI

Fiona’s 命令行界面工具“fio”, 文档描述在 docs/cli.rst. 下面使用fio info列出数据文件的信息。

    $ fio info docs/data/test_uk.shp
    { 'bbox': (-8.621389, 49.911659, 1.749444, 60.844444),
      'count': 48,
      'crs': { u'datum': u'WGS84', u'no_defs': True, u'proj': u'longlat'},
      'driver': u'ESRI Shapefile',
      'schema': { 'geometry': 'Polygon',
                  'properties': OrderedDict([(u'CAT', 'float:16'), (u'FIPS_CNTRY', 'str:80'), (u'CNTRY_NAME', 'str:80'), (u'AREA', 'float:15.2'), (u'POP_CNTRY', 'float:15.2')])}}

## 安装Fiona

Fiona 要求 Python 2.6, 2.7, 3.3, 或 3.4 和 GDAL/OGR 1.8以上版本。从源代码编译需要 C 编译器以及 GDAL 和 Python 开发头文件和支持库文件 (libgdal1-dev for Debian/Ubuntu, gdal-dev for CentOS/Fedora).

在OS X上，Kyngchaos GDAL frameworks 可以提供GDAL/OGR在OS X上的依赖软件包，或使用Homebrew (brew install gdal) 进行安装。

* Python Requirements

Fiona 依赖于模块 six 和 argparse。Easy_install 和 pip 将自动下载和安装依赖文件，但Windows用户可能需要单独下载安装。

* Unix-like systems

如果正在使用 virtualenv (如果没有, 跳过下面的4个命令) ，GDAL/OGR libraries, headers, 和 gdal-config 等程序都安装在缺省的位置:

```
    $ mkdir fiona_env
    $ virtualenv fiona_env  
    $ source fiona_env/bin/activate  
    (fiona_env)$ pip install Fiona
```

如果 gdal-config 不可用或者 GDAL/OGR 头和库文件没有安装在缺省位置就必须自行设定include dirs, library dirs, 和 libraries选项，在setup.cfg 文件或者 setup 命令行设置(使用 git)，如下所示:

(fiona_env)$ git clone git://github.com/Toblerity/Fiona.git
(fiona_env)$ cd Fiona
(fiona_env)$ python setup.py build_ext -I/path/to/gdal/include -L/path/to/gdal/lib -lgdal install

* Windows

二进制安装包在 http://www.lfd.uci.edu/~gohlke/pythonlibs/#fiona 或者通过 PyPI 获得。

## Development and testing

从源码构建需要Cython，测试需要Nose。如果GDAL/OGR libraries, headers, 和 gdal-config 程序安装在缺省位置，如下方式安装:

```
    (fiona_env)$ git clone git://github.com/Toblerity/Fiona.git
    (fiona_env)$ cd Fiona
    (fiona_env)$ python setup.py develop
    (fiona_env)$ nosetests
```

如果是一个非标准的环境，需要制定 include 和 lib 目录以及 GDAL library目录。编译命令行如下：

(fiona_env)$ python setup.py build_ext -I/path/to/gdal/include -L/path/to/gdal/lib -lgdal develop
(fiona_env)$ nosetests